# Import Packages

In [ ]:
# Importing necessary libraries
import torch
import gc
import torch.nn as nn
import torch.nn.functional as F

import math
import numpy as np
import seaborn as sns
import scienceplots
import matplotlib.pyplot as plt

import traceback
from collections import OrderedDict
from tqdm.auto import tqdm

# Set the random seed for reproducibility
seed = 20230808
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Set the device to GPU if available, otherwise use CPU
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

# Perform garbage collection and empty the GPU cache in PyTorch
gc.collect()
torch.cuda.empty_cache()


# Model

In [ ]:
class NN(nn.Module):
    """
    A simple neural network class.

    Args:
        input_size (int): The number of input features.
        hidden_size (int): The number of hidden units in each hidden layer.
        output_size (int): The number of output features.
        depth (int): The number of hidden layers.
        ac (torch.nn.Module): The activation function to use for each hidden layer.
    """

    def __init__(
        self,
        input_size,
        hidden_size,
        output_size,
        depth,
        ac=torch.nn.Tanh,
    ):
        super(NN, self).__init__()

        layers = [('input', torch.nn.Linear(input_size, hidden_size))]
        layers.append(('input_activation', ac()))
        for i in range(depth):
            layers.append(
                ('hidden_%d' % i, torch.nn.Linear(hidden_size, hidden_size))
            )
            layers.append(('activation_%d' % i, ac()))
        layers.append(('output', torch.nn.Linear(hidden_size, output_size)))

        layerDict = OrderedDict(layers)
        self.layers = torch.nn.Sequential(layerDict)

    def forward(self, x):
        """
        Forward pass through the network.

        Args:
            x (torch.Tensor): The input tensor of shape (batch_size, input_size).

        Returns:
            torch.Tensor: The output tensor of shape (batch_size, output_size).
        """
        return self.layers(x)


# Training Loop

A one-dimensional wave equation is chosen for our experiments, which, in mathematical form, is defined as follows:

$$ u_{tt} - u_{xx} = 0$$

for this wave equation, its initial conditions and the homogeneous Dirichlet boundary conditions are given, as follows:

$$ u(0, x) = \frac{1}{2} \sin (\pi x)$$
$$ u_t(0, x) = \pi \sin (3 \pi x)$$
$$ u(t, 0) = 0 $$
$$ u(t, 0) = 0 $$

In [ ]:
def init_weights(layer):
    """
    Initializes the weights of a layer with Xavier normal initialization.
    Args:
        layer (torch.nn.Module): The layer to initialize.
    """
    if isinstance(layer, nn.Linear):
        torch.nn.init.xavier_normal_(layer.weight)


class Trainer:
    def __init__(self):
        # deep neural networks
        self.iter = 0
        self.model = NN(
            input_size=2,
            hidden_size=100,
            output_size=1,
            depth=5,
            ac=torch.nn.Tanh
        ).to(device)

        # use the Glorot normal initializer for initialization
        self.model.apply(init_weights)

        # The initial conditions, boundary conditions
        # with Nu approximating 300
        self.h = 0.1
        self.k = 0.1
        x_u = torch.arange(0, 1 + self.h, self.h)
        t_u = torch.arange(0, 1 + self.k, self.k)
        c0 = torch.stack(torch.meshgrid(x_u, t_u[0])).reshape(2, -1).T
        c1 = torch.rand(300, 2)
        c2 = torch.stack(torch.meshgrid(x_u[0], t_u)).reshape(2, -1).T
        c3 = torch.stack(torch.meshgrid(x_u[-1], t_u)).reshape(2, -1).T

        y_c0 = 1/2*torch.sin(math.pi * c0[:, 0])
        y_c1 = 1/2*torch.sin(math.pi*c1[:, 0])*torch.cos(math.pi*c1[:, 1])+1/3*torch.sin(3*math.pi*c1[:, 0])*torch.sin(3*math.pi*c1[:, 1])
        y_c2 = torch.zeros(len(c2))
        y_c3 = torch.zeros(len(c3))

        self.X_u = torch.cat([c0, c1, c2, c3]).to(device)
        self.y_u = torch.cat([y_c0, y_c1, y_c2, y_c3]).unsqueeze(1).to(device)
        print(f'N_u is {self.X_u.shape[0]}')

        # Data in the space-time domain
        # with Nf approximating 40000
        self.h = 0.1
        self.k = 0.1
        x_f1 = torch.arange(0, 1 + self.h, self.h)
        t_f1 = torch.arange(0, 1 + self.k, self.k)
        X_f1 = torch.stack(torch.meshgrid(x_f1, t_f1)).reshape(2, -1).T
        X_f2 = torch.rand(40000, 2)
        self.X_f = torch.cat([X_f1, X_f2]).to(device)
        self.X_f.requires_grad = True
        print(f'N_f is {self.X_f.shape[0]}')

        # General Loss Function
        self.criterion = torch.nn.MSELoss(reduction='mean')

        # Two Optimizer
        self.optimizer = torch.optim.LBFGS(
            self.model.parameters(),
            lr=1, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe",
            )

        self.adam = torch.optim.Adam(self.model.parameters(), 
                                     lr=6e-6, 
                                     weight_decay=1e-6
                                    )
        self.boss_loss = math.inf

    def loss_func(self):
        self.adam.zero_grad()
        self.optimizer.zero_grad()

        # loss using observations of initial and boundary conditions
        loss_u = self.criterion(self.model(self.X_u), self.y_u)

        # loss based on partial differential equations
        u = self.model(self.X_f)
        u_X = torch.autograd.grad(
            u, self.X_f,
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]

        u_x, u_t = u_X[:, 0], u_X[:, 1]

        u_xx = torch.autograd.grad(
            u_x, self.X_f,
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0][:, 0]

        u_tt = torch.autograd.grad(
            u_t, self.X_f,
            grad_outputs=torch.ones_like(u_t),
            retain_graph=True,
            create_graph=True
        )[0][:, 1]

        loss_f = self.criterion(u_tt - u_xx, u_xx*0)
        loss = loss_u + loss_f        
        loss.backward()
        if self.iter % 100 == 0:
            if loss < self.boss_loss:
                self.boss_loss = loss
                torch.save(self.model.state_dict(), 'boss_model.ckpt')
                print(
                    'Iter %d, Loss: %.5e, Loss_u: %.5e, Loss_f: %.5e with lowest loss %.5e' % (
                        self.iter, loss.item(), loss_u.item(), loss_f.item(), self.boss_loss)
                )
            
        self.iter = self.iter + 1
        return loss

    def train(self):
        self.model.train()
        sesese

# Training

In [ ]:
trainer = Trainer()
trainer
trainer.optimizer.step(trainer.loss_func)

# Evaluation

In [ ]:
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import scienceplots
import plotly.express as px

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# plt.style.use(['ipynb', 'use_mathtext', 'colors5-light', 'science'])
# sns.set_style('whitegrid')
# sns.set_palette('RdBu')
# sns.set(
#     rc={'text.usetex': True},
#     font='serif',
#     font_scale=1.2
# )

# matplotlib.rcParams['font.sans-serif'] = ['SimHei']
# matplotlib.rcParams['font.serif'] = ['SimHei']
# sns.set_style('darkgrid', {'font.sans-serif': ['simhei', 'Arial']})


In [ ]:
h = 0.001
k = 0.001
x = torch.arange(0, 1, h)
t = torch.arange(0, 1, k)
X = torch.stack(torch.meshgrid(x, t)).reshape(2, -1).T
X = X.to(device)


In [ ]:
def ground_truth(X):
    x = X[:, 0]
    t = X[:, 1]
    y = 1/2*torch.sin(torch.pi*x)*torch.cos(torch.pi*t)+1 / \
        3*torch.sin(3*torch.pi*x)*torch.sin(3*torch.pi*t)
    return y


In [ ]:
model = NN(input_size=2,
           hidden_size=100,
           output_size=1,
           depth=5,
           ac=torch.nn.Tanh).to(device)
model.load_state_dict(torch.load('boss_model.ckpt'))
model.eval()
with torch.no_grad():
    # y_pred = ground_truth(X).reshape(len(x), len(t)).detach().cpu()
    y_pred = model(X).reshape(len(x), len(t)).detach().cpu().numpy()


In [ ]:
sns.heatmap(y_pred, cmap='jet')
plt.show()


In [ ]:
y_pred.shape


In [ ]:
x = torch.arange(0, 1, h).unsqueeze(dim=1).to(device)
t = (torch.ones(x.shape)*0.2).to(device)

plt.scatter(x.detach().cpu(), model(torch.cat([x, t], dim=1)).detach().cpu())
